In [16]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [17]:
import pandas as pd
from gensim.utils import simple_preprocess
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the Excel file with the correct encoding
file_path = '/content/chatwith pdf.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Preprocess the text data (tokenization)
chatbot_responses = df['RAG chatbot response'].tolist()
human_responses = df['Human Response'].tolist()

# Load a pre-trained BERT model from sentence-transformers
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Get BERT embeddings for each response
chatbot_embeddings = model.encode(chatbot_responses)
human_embeddings = model.encode(human_responses)

# Compute cosine similarity between corresponding chatbot and human response embeddings
cosine_similarities = np.array([cosine_similarity([chatbot_embeddings[i]], [human_embeddings[i]])[0, 0]
                                for i in range(len(chatbot_embeddings))])

# Normalize similarity scores to a scale of 1 to 5
normalized_similarity_scores = np.interp(cosine_similarities, (cosine_similarities.min(), cosine_similarities.max()), (1, 5))

# Add the BERT similarity scores to the dataframe
df['BERT Similarity Score'] = normalized_similarity_scores

# Save the updated dataframe to a new CSV file
df.to_csv("Output1.csv", index=False)

print(df.head())


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

                                               Query  \
0  What is the maximum value you will pay if my c...   
1                Who is covered to drive other cars?   
2                   What if I leave my car unlocked?   
3            Does Churchill have approved repairers?   
4                                 What is DriveSure?   

                                RAG chatbot response Category    Type  \
0   Where damage to your car is covered under you...     What  Direct   
1   Your certificate of motor insurance will show...      Who  Direct   
2  We won’t pay a claim for theft or attempted th...     What  Direct   
3  Churchill customers have access to a national ...     What  Direct   
4  DriveSure is our telematics insurance product....     What  Direct   

                                      Human Response  BERT Similarity Score  
0  We’ll pay the cost of repairing or replacing y...               4.062111  
1  Your certificate of motor insurance will show ...               3

In [20]:

count_over_3_5 = np.sum(normalized_similarity_scores > 3.0)


total_count = len(normalized_similarity_scores)


accuracy = count_over_3_5 / total_count

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.77
